In [1]:
%pip install pypdf2
%pip install -U langchain 
%pip install -U langchain_openai
%pip install -U langchain[all]
%pip install -U chromadb
%pip install -U pydantic 

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.9
    Uninstalling langchain-0.3.9:
      Successfully uninstalled langchain-0.3.9

[notice] A new release of pip is available: 24.0 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.8 MB/s eta 0:00:0

In [ ]:
# Import necessary libraries
import os
from PyPDF2 import PdfReader
from langchain_openai import AzureOpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.vectorstores import Chroma
from langchain_openai import AzureOpenAI
import time


In [4]:
import sys
sys.path.insert(1, '../')
import init_creds as creds
 
AZURE_OPENAI_API_KEY = creds.get_api_key()
AZURE_OPENAI_ENDPOINT = creds.get_endpoint()
 
if not AZURE_OPENAI_API_KEY:
    raise ValueError("No AZURE_OPENAI_API_KEY set for Azure OpenAI API")
if not AZURE_OPENAI_ENDPOINT:
    raise ValueError("No AZURE_OPENAI_ENDPOINT set for Azure OpenAI API")

os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_VERSION"] = "2024-07-01-preview"
os.environ["AZURE_OPENAI_API_VERSION"] = os.environ["OPENAI_API_VERSION"]
print(f'Using Azure OpenAI API key: {os.environ["AZURE_OPENAI_API_KEY"]}')

Using Azure OpenAI API key: b0dcf29cef014c0fa984a0feab1ee9aa


In [5]:
# Sample Chat Completion using Azure OpenAI API


client = AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=os.environ["OPENAI_API_VERSION"] 
)

In [9]:
# Step 1: Load and Chunk Documents from multiple PDFs
def load_pdfs_from_directory(directory_path):
    documents = []
    for filename in os.listdir(directory_path):
        if filename.endswith('.pdf'):
            file_path = os.path.join(directory_path, filename)
            pdf_reader = PdfReader(file_path)
            for page in pdf_reader.pages:
                text = page.extract_text()
                if text:
                    documents.append(text)
    return documents

In [10]:
# Specify the directory containing your PDF files
pdf_directory = "data"
raw_documents = load_pdfs_from_directory(pdf_directory)

In [11]:
# Split documents into smaller chunks for better processing
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.create_documents(raw_documents)
print(len(chunks))

269


In [16]:
# Step 2: Create Embeddings and Store in Vector Database
# Initialize the embedding model from OpenAI
embedding_model = AzureOpenAIEmbeddings(model="text-embedding-3-large", azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
#embedding_model = AzureOpenAIEmbeddings(model="all-MiniLM-L6-v2", azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"] )

In [17]:
# Create a vector store using Chroma to hold the document chunks and their embeddings
vector_store = Chroma(embedding_function = embedding_model)
for chunk in chunks[0:50]:
    vector_store.add_documents([chunk])
    time.sleep(1)

# Note: This cell will take at least a minute to run because of the request throttling

KeyboardInterrupt: 

In [11]:
## Step 3: Set Up the Retriever
# Create a retriever that will fetch relevant chunks based on user queries
retriever = vector_store.as_retriever()

In [12]:
# Step 4: Define the Prompt Template
# Create a structured prompt template for the language model to follow
template = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.

Question: {question}
Context: {context}

Answer:
"""

In [13]:
# Initialize the prompt with the defined template
prompt = ChatPromptTemplate.from_template(template)

In [19]:
# Step 5: Build the RAG Chain
llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment="gpt-4o-mini",
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0
)

In [20]:
# Construct the RAG chain that combines retrieval and generation processes
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [21]:
# Step 6: Invoke the RAG Chain with a Query

def answer_queries_using_rag(query):
    response = ""
    try:
        response = rag_chain.invoke(query)
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    return response

In [ ]:
query = "What are the main topics discussed in the documents?"
print(answer_queries_using_rag(query))